<div><svg width="600" height="150" viewBox="0 0 600 150" xmlns="http://www.w3.org/2000/svg">
  <defs>
    <linearGradient id="kaviGrad" x1="0%" y1="0%" x2="100%" y2="100%">
      <stop offset="0%" style="stop-color:#1e3a8a;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#3b82f6;stop-opacity:1" />
    </linearGradient>
    <filter id="glowText">
      <feGaussianBlur stdDeviation="1.5" result="coloredBlur"/><feMerge><feMergeNode in="coloredBlur"/><feMergeNode in="SourceGraphic"/></feMerge>
    </filter>
  </defs>
  <rect width="600" height="150" fill="#0f172a" rx="20"/>
  <text x="50" y="90" font-family="Verdana, sans-serif" font-size="60" font-weight="bold" fill="url(#kaviGrad)" filter="url(#glowText)">kavi.ai</text>
  <text x="300" y="90" font-family="Verdana, sans-serif" font-size="24" fill="#94a3b8">by machha kiran</text>
  <line x1="50" x2="550" y1="110" y2="110" stroke="#334155" stroke-width="2"/>
  <circle cx="560" cy="40" r="10" fill="#3b82f6"/>
  <circle cx="530" cy="30" r="6" fill="#1e40af"/>
</svg></div>

<img src="../01-language-model-and-the-basics-of-training/header_banner.png" width="100%" alt="Banner">

# ⚡ Performance Optimization & Robustness

Welcome to this training session. In this notebook, we will explore advanced LLM training techniques.

---

In [ ]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer
!pip install -U flash-attn


In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=NEFTUNE
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

In [ ]:
import wandb
wandb.login()

In [ ]:
import sys
sys.path.append('/root/llm-training-course/')

In [ ]:
from datasets import load_dataset
train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:10%]","train[10%:15%]"])

In [ ]:
train_ds

In [ ]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"] + "\n" +  x["chosen"][0]["content"]}] + x["chosen"][1:] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"]+ "\n"+ x["chosen"][0]["content"]}]+ x["chosen"][1:] })

In [ ]:
columns_to_remove = [c for c in train_ds.column_names if c not in ["messages"]]
train_ds = train_ds.remove_columns(columns_to_remove)

columns_to_remove = [c for c in eval_ds.column_names if c not in ["messages"]]
eval_ds = eval_ds.remove_columns(columns_to_remove)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    attn_implementation="flash_attention_2"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from helpers import set_padding_for_tokenizer
set_padding_for_tokenizer(tokenizer)

In [ ]:
print(model)

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],           
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
from peft import get_peft_model
peft_model = get_peft_model(model, peft_config)

## Step 1: Memory Optimization

### **Purpose:**
Trading computation for memory to train larger models or use larger batches.

### **Line-by-Line Breakdown:**
- `gradient_checkpointing`: Save memory by re-computing activations.

In [ ]:
import os
from trl import SFTConfig, SFTTrainer

args = SFTConfig(
    output_dir=os.getenv("WANDB_RUN_ID"),
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=1,
    lr_scheduler_type="constant",
    bf16=True,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.1,
    max_grad_norm=.3,
    learning_rate=5e-5
)


In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    neftune_noise_alpha=5
)
trainer.train()

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    neftune_noise_alpha=5
)
trainer.train()